# Jupyter Notebook - KoBART 요약기 프로그램
# 기계학습 미니 프로젝트
- #### 201904022 김상옥
- #### 202004064  소재현
---

# 코드 설명

KoBART 모델을 활용하여 PDF 파일에서 텍스트를 추출 후 해당 텍스트를 요약하는 프로그램.

#### 학습과 해당 코드에서 사용한 라이브러리
```
!pip install pandas
!pip install torch==2.0.1
!pip install transformers==4.32.1
!pip install tokenizers==0.13.3
!pip install lightning==2.0.8
!pip install streamlit==1.26.0
!pip install wandb==0.15.9
!pip install loguru
!pip install rouge_score

!pip install threading
!pip install tkinter
!pip install re
!pip install PyMuPDF
```



## 주요 기능
1. **PDF 파일 로드**: 
   - 사용자가 PDF 파일을 선택하면, PyMuPDF(`fitz` 라이브러리)을 사용하여 PDF에서 텍스트를 추출합니다. 
   - 추출된 텍스트는 Tkinter의 텍스트 상자에 표시됩니다.

2. **한국어만 추출**:
   - 입력된 텍스트에서 한글만 남기고, 영어, 숫자, 특수문자 등은 제거됩니다. 이는 정규 표현식(`re.sub()`)을 사용하여 처리됩니다.

3. **요약 생성**:
   - 사용자가 "요약문 생성" 버튼을 클릭하면, KoBART 모델을 사용하여 텍스트 요약을 생성합니다.
   - 최대 길이(`max_length`)와 최소 길이(`min_length`)를 사용자가 지정할 수 있으며, 이 값에 따라 요약이 생성됩니다.
   - 요약이 진행되는 동안 "요약 생성 중..." 메시지가 표시되며, 요약 완료 후 결과는 출력 창에 표시됩니다.

4. **멀티스레딩 처리**:
   - 요약 과정은 별도의 스레드에서 실행되어 GUI가 멈추지 않도록 처리됩니다.

## 코드 구조
1. **라이브러리 로드**:
   - `torch`, `transformers`, `fitz` 등 다양한 라이브러리를 사용하여 모델 로딩, PDF 읽기, 텍스트 요약 등을 처리합니다.

2. **모델 및 토크나이저 로드**:
   - `PreTrainedTokenizerFast`와 `BartForConditionalGeneration`을 사용하여 KoBART 모델과 토크나이저를 로드합니다.

3. **GUI 구성**:
   - `Tkinter`를 사용하여 기본적인 GUI를 구현합니다.
   - 원문 입력 창, 출력 창, 최대/최소 길이 입력 창, 버튼들이 포함됩니다.

4. **요약 함수**:
   - `summarize_text` 함수는 KoBART 모델을 사용하여 텍스트를 요약합니다.
   - `summarize_and_update_button` 함수는 요약이 끝난 후, GUI의 출력 창에 결과를 업데이트합니다.

5. **PDF 텍스트 추출 함수**:
   - `extract_text_from_pdf` 함수는 PDF 파일에서 텍스트를 추출하고, 이를 반환합니다.

## 사용법
- **PDF 파일 불러오기**: "PDF 파일 불러오기" 버튼을 클릭하여 PDF 파일을 선택하면, 파일에서 텍스트가 자동으로 추출되어 입력창에 표시됩니다.
- **요약문 생성**: "요약문 생성" 버튼을 클릭하면 입력된 텍스트를 요약하며, 결과는 출력창에 나타납니다.


# 각 라이브러리 및 토크나이저, 모델 불러오기

In [60]:
import torch # 2.0.1
from transformers import PreTrainedTokenizerFast # 4.32.1
from transformers.models.bart import BartForConditionalGeneration

from tkinter import scrolledtext, filedialog
import sys
import tkinter as tk 
import fitz  # PDF 읽기
import threading  # 스레딩 모듈 
import re # 정규화

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('./model/tokenizer')
model = BartForConditionalGeneration.from_pretrained('./model/model')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# input_text를 요약하여 요약 텍스트를 반환하는 함수

In [61]:
def summarize_text(input_text, max_length=150, min_length=40, num_beams=5, length_penalty=1.2, early_stopping=True):
    # 입력 텍스트를 토큰화하여 인코딩
    input_ids = tokenizer.encode(       # 입력 텍스트를 단어 ID로 변환
        input_text,                     # 입력 텍스트
        return_tensors="pt",            # pytorch로 변환 -> 모델에 입력하기 위해
        max_length=1024,                # 최대 길이 1024
        truncation=True                 # 최대 길이가 초과되면 텍스트 자르기
    )

    # 모델을 사용하여 요약 생성
    summary_ids = model.generate(       # 문장 생성 함수 
        input_ids,                      # 인코드한 변수
        max_length=max_length,          # 요약문 최대 길이
        min_length=min_length,          # 요약문 최소 길이
        length_penalty=length_penalty,  # 요약문의 길이 설정 -> 값이 높을수록 길이가 짧아짐
        num_beams=num_beams,            # 단어 후보의 수 -> 단어 후보 4개에서 하나를 선택 -> 클수록 정확도 향상 
        early_stopping=early_stopping   # 더 이상 생성되는 단어가 없으면 일찍 종료할 수 있는 기능
    )

    # 요약 결과 디코딩
    summary_text = tokenizer.decode(    # 단어 ID를 다시 텍스트로 변환
        summary_ids[0],                 # 요약문에 해당하는 인덱스
        skip_special_tokens=True        # 의미 있는 텍스트만 변환함 
    )        
    
    return summary_text                 # 최종 요약 문장

# 파일 경로에 PDF를 열고 텍스트를 추출하여 반환하는 함수

In [62]:
# PDF 파일에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    # PyMuPDF로 PDF 열기
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text("text")  # 텍스트 추출
    return text

# 요약 생성기 GUI 함수
- #### 윈도우 사이즈 800 x 900
- #### 구성요소
- - 원문 입력  (요약 전 원문을 입력 및 표시되는 창)
- - 예측 요약문 (최종적으로 요약된 텍스트가 표시되는 창)
- - 최대, 최소 길이 입력, num_beams, length_penalty, early_stopping (생성 파라미터 제어창
- - PDF 불러오기 버튼 (파일 탐색기를 이용하여 PDF를 불러오는 버튼)
- - 테스트 원문 생성 (테스트 원문을 입력창에 입력하는 버튼)
- - 요약문 생성 (원문 입력에 있는 텍스트를 요약하는 버튼)
- - 설정 초기화 (각 파라미터 값 초기화)

- #### `on_load_pdf_button_click()` : pdf 불러오기 버튼을 누를 시 실행
- - pdf를 불러와서 텍스트를 추출하고, 입력창에 텍스트를 표시함

- #### `on_summarize_button_click` : 요약문 생성 버튼을 누를 시 실행
- - 입력창에 있는 텍스트를 가져와서 영어, 특수문자를 제거하여 한글만 추출
- - `summarize_text`에 text를 넣어 요약문을 생성한 후 최종적으로 출력창에 표시함

In [63]:
test_text = """인공지능은 현대 사회에서 가장 주목받는 기술 중 하나로, 다양한 산업 분야에 걸쳐 혁신을 일으키고 있습니다. 특히, 의료, 금융, 제조업, 교통, 교육 등 여러 분야에서 인공지능이 적용되면서 효율성과 정확성이 향상되고 있습니다. 의료 분야에서는 인공지능 기반의 진단 시스템이 환자의 데이터를 분석하여 질병을 조기에 발견하고, 개인 맞춤형 치료법을 제안하는 데 중요한 역할을 하고 있습니다. 예를 들어, 암 진단 과정에서 방대한 양의 의료 영상을 분석하여 종양의 위치와 크기를 정확히 파악하고, 이를 기반으로 최적의 치료 계획을 수립할 수 있습니다. 이러한 기술은 의료 전문가의 업무를 보조하며, 환자의 생존율을 높이는 데 기여하고 있습니다.

금융 분야에서도 인공지능의 도입이 활발히 이루어지고 있습니다. 자동화된 거래 시스템은 시장 데이터를 실시간으로 분석하고, 최적의 투자 전략을 제안함으로써 투자자들이 빠르게 변화하는 시장 상황에 효과적으로 대응할 수 있도록 돕습니다. 또한, 금융 사기 탐지 시스템은 비정상적인 거래 패턴을 실시간으로 감지하여 사기 행위를 예방하는 데 중요한 역할을 하고 있습니다. 이외에도, 고객 상담 서비스에서 챗봇 기술이 도입되어 24시간 고객 응대가 가능해졌으며, 이를 통해 고객 만족도가 크게 향상되고 있습니다.

제조업에서는 스마트 공장이 인공지능 기술을 활용하여 생산 공정을 최적화하고 있습니다. 로봇과 자동화 시스템이 결합된 스마트 공장은 실시간 데이터를 수집하고 분석하여 생산 라인의 효율성을 극대화합니다. 이러한 시스템은 예측 유지보수 기술을 통해 기계의 고장을 미리 감지하고, 필요한 수리를 사전에 수행하여 생산 중단을 최소화할 수 있습니다. 결과적으로 생산 비용이 절감되고, 제품 품질이 향상되어 경쟁력을 강화할 수 있습니다.

교통 분야에서도 자율주행 차량과 같은 첨단 기술이 주목받고 있습니다. 자율주행 차량은 센서와 인공지능 알고리즘을 활용하여 주변 환경을 분석하고, 최적의 경로를 선택하여 안전하고 효율적인 주행을 가능하게 합니다. 이는 교통 혼잡을 줄이고, 교통 사고 발생률을 낮추는 데 큰 기여를 할 것으로 기대됩니다. 또한, 대중교통 시스템에서도 인공지능이 적용되어 승객 수요를 예측하고, 최적의 운행 스케줄을 제안함으로써 운영 효율성을 높이고 있습니다.

교육 분야에서는 인공지능이 학습자의 데이터를 분석하여 개인 맞춤형 학습 프로그램을 제공하고 있습니다. 이러한 프로그램은 학습자의 수준과 선호도에 맞게 학습 내용을 조정하여 효과적인 학습을 지원합니다. 특히, 온라인 교육 플랫폼에서는 인공지능 튜터가 학생들의 학습 진도를 실시간으로 모니터링하고, 필요한 피드백을 제공함으로써 학습 성과를 향상시키고 있습니다. 또한, 자연어 처리 기술을 활용한 언어 학습 애플리케이션은 사용자가 실시간으로 언어 능력을 연습하고 개선할 수 있도록 지원합니다.

결론적으로, 인공지능은 다양한 분야에서 우리의 삶을 변화시키고 있으며, 앞으로도 그 영향력은 더욱 커질 것으로 보입니다. 이러한 변화는 새로운 기회를 창출함과 동시에 사회적, 윤리적 도전 과제를 제기하기도 합니다. 따라서 인공지능 기술의 발전과 함께 책임 있는 개발과 사용이 필요하며, 이를 위해 관련 법률과 정책이 마련되어야 할 것입니다. 미래 사회에서 인공지능이 긍정적인 역할을 지속적으로 수행하기 위해서는 기술적 혁신과 더불어 사회적 합의가 뒷받침되어야 할 것입니다.
"""

def create_gui():
    window = tk.Tk()
    window.title("기계학습 미니 프로젝트 KoBART PDF 요약기")
    window.geometry("800x900")

    label_input = tk.Label(window, text="원문", font=("Arial", 14))
    label_input.pack(pady=10)

    input_textbox = scrolledtext.ScrolledText(window, width=80, height=13, font=("Arial", 12))
    input_textbox.pack(pady=10)

    label_output = tk.Label(window, text="예측 요약문", font=("Arial", 14))
    label_output.pack(pady=10)

    output_textbox = scrolledtext.ScrolledText(window, width=80, height=7, font=("Arial", 12))
    output_textbox.pack(pady=10)

    def create_slider(label_text, from_, to_, default):
        frame = tk.Frame(window)
        frame.pack(pady=5, padx=20, anchor="w")
        label = tk.Label(frame, text=label_text, font=("Arial", 12))
        label.pack(side=tk.LEFT)
        slider = tk.Scale(frame, from_=from_, to=to_, orient=tk.HORIZONTAL, length=300)
        slider.set(default)
        slider.pack(side=tk.RIGHT)
        return slider

    max_length_slider = create_slider("최대 길이 (기본 150): ", 20, 1000, 150)
    min_length_slider = create_slider("최소 길이 (기본 40): ", 10, 200, 40)
    num_beams_slider = create_slider("빔 수 (기본 5): ", 1, 10, 5)
    
    # 길이 패널티 슬라이더 생성 (소수점 조절 가능)
    length_penalty_frame = tk.Frame(window)
    length_penalty_frame.pack(pady=5, padx=20, anchor="w")

    length_penalty_label = tk.Label(
        length_penalty_frame,
        text="길이 패널티 (기본 1.2):",
        font=("Arial", 12)
    )
    length_penalty_label.pack(side=tk.LEFT)  # 라벨을 왼쪽 정렬

    length_penalty_slider = tk.Scale(
        length_penalty_frame,
        from_=-5.0,        # 최소 값
        to=5.0,            # 최대 값
        resolution=0.1,    # 소수점 단위 (0.1씩 증가)
        orient=tk.HORIZONTAL,
        length=250         # 슬라이더 길이 조정
    )
    length_penalty_slider.set(1.2)  # 기본값 설정
    length_penalty_slider.pack(side=tk.RIGHT)  # 슬라이더를 오른쪽 정렬


    early_stopping_var = tk.BooleanVar(value=True)
    early_stopping_checkbox = tk.Checkbutton(window, text="Early Stopping 활성화", variable=early_stopping_var, font=("Arial", 12))
    early_stopping_checkbox.pack(pady=10)

    def on_load_pdf_button_click():
        file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
        if file_path:
            extracted_text = extract_text_from_pdf(file_path)
            input_textbox.delete(1.0, tk.END)
            input_textbox.insert(tk.END, extracted_text)

    def on_summarize_button_click():
        input_text = input_textbox.get("1.0", tk.END).strip()
        korean_text = re.sub(r'[^가-힣\s]', '', input_text)
        input_textbox.delete(1.0, tk.END)
        input_textbox.insert(tk.END, korean_text)

        if not korean_text:
            output_textbox.delete(1.0, tk.END)
            output_textbox.insert(tk.END, "입력된 텍스트에 한국어가 없습니다.")
            return

        max_length = int(max_length_slider.get())
        min_length = int(min_length_slider.get())
        num_beams = int(num_beams_slider.get())
        length_penalty = float(length_penalty_slider.get())
        early_stopping = early_stopping_var.get()

        output_textbox.delete(1.0, tk.END)
        output_textbox.insert(tk.END, "요약 생성 중...")
        summarize_button.config(bg="yellow")

        threading.Thread(
            target=summarize_and_update_button,
            args=(korean_text, max_length, min_length, num_beams, length_penalty, early_stopping, output_textbox),
            daemon=True
        ).start()

    def summarize_and_update_button(input_text, max_length, min_length, num_beams, length_penalty, early_stopping, output_textbox):
        summary = summarize_text(input_text, max_length, min_length, num_beams, length_penalty, early_stopping).replace('.', '.\n')
        window.after(0, update_output_text, summary)

    def update_output_text(summary):
        output_textbox.delete(1.0, tk.END)
        output_textbox.insert(tk.END, summary)
        summarize_button.config(bg="green")

    def reset_settings():
        max_length_slider.set(150)
        min_length_slider.set(40)
        num_beams_slider.set(5)
        length_penalty_slider.set(1.2)
        early_stopping_var.set(True)
        output_textbox.delete(1.0, tk.END)
        output_textbox.insert(tk.END, "설정이 초기화되었습니다.")

    def test_settings():
        input_textbox.delete(1.0, tk.END)
        input_textbox.insert(tk.END, test_text)
        
    button_frame = tk.Frame(window)
    button_frame.pack(pady=10)
    
    load_pdf_button = tk.Button(button_frame, text="PDF 파일 불러오기", font=("Arial", 14), command=on_load_pdf_button_click)
    load_pdf_button.pack(side=tk.LEFT, padx=10)
    
    test_button = tk.Button(button_frame, text="테스트 원문 생성", font=("Arial", 14), command=test_settings)
    test_button.pack(side=tk.LEFT, padx=10)
    
    summarize_button = tk.Button(button_frame, text="요약문 생성", font=("Arial", 14), command=on_summarize_button_click)
    summarize_button.pack(side=tk.LEFT, padx=10)

    reset_button = tk.Button(button_frame, text="설정 초기화", font=("Arial", 14), command=reset_settings)
    reset_button.pack(side=tk.RIGHT, padx=10)

    window.mainloop()

In [68]:
if __name__ == "__main__":
    create_gui()